# Discord Data Cleaning and Processing

This notebook processes raw Discord messages, extracting ticker symbols, performing sentiment analysis, and preparing the data for use in portfolio analysis.

In [1]:
import pandas as pd, textwrap
import json
import os
import sys
import re
from datetime import datetime
import ast
import json
from langchain import OpenAI, LLMChain, PromptTemplate
from pathlib import Path
#install textblob if not already installed
from textblob import TextBlob #pip install textblob
from dotenv import load_dotenv
load_dotenv()

RAW  = Path('../data/raw/discord_msgs.csv')
OUT  = Path('../data/processed/discord_msgs_clean.parquet')


## Data Cleaning Process

We'll perform the following steps:
1. Load raw Discord messages and normalize timestamps
2. Apply feature engineering to extract useful information
3. Perform sentiment analysis using TextBlob
4. Save the cleaned data in an efficient format (Parquet)

In [2]:
# ------------------------------------------------------------------ #
# 1. LOAD & BASIC NORMALISATION
df = pd.read_csv(RAW)
df['created_at'] = pd.to_datetime(df['created_at'], utc=True)
df = df.sort_values('created_at')              # chronological
df = df.drop_duplicates('message_id')          # safety

# ------------------------------------------------------------------ #
# 2. LIGHT FEATURE ENGINEERING
df['char_len']   = df['content'].str.len()
df['word_len']   = df['content'].str.split().str.len()

# Tickers → list ( [] if NaN )
df['tickers'] = (df['tickers_detected']
                   .fillna('')
                   .apply(lambda s: re.findall(r'\$[A-Z]{2,6}', s)))

# Tweet URLs → list
df['tweet_urls'] = (df['tweet_urls']
                      .fillna('')
                      .str.split(',\s*') )

# Basic sentiment   (polarity ∈ [-1,1])
df['sentiment'] = df['content'].apply(
    lambda t: TextBlob(str(t)).sentiment.polarity
)

# Command flag (e.g. “!history”)
df['is_command'] = df['content'].str.startswith('!')

# Keep only useful columns
keep = ['message_id','created_at','channel','author_name',
        'content','tickers','tweet_urls',
        'char_len','word_len','sentiment','is_command']
df = df[keep]

# ------------------------------------------------------------------ #
# 3. SAVE TIDY VERSION
df.to_parquet(OUT, index=False)
print(f'✅ cleaned file saved → {OUT}\n'
      f'   rows: {len(df):,}   cols: {len(df.columns)}')

✅ cleaned file saved → ..\data\processed\discord_msgs_clean.parquet
   rows: 119   cols: 11


## Data Exploration

Let's examine a sample of our processed data to verify the cleaning steps worked correctly.

In [ ]:
# Load the cleaned data and display a sample
clean_df = pd.read_parquet(OUT)
clean_df.head()

## Ticker Symbol Analysis

Let's analyze the most frequently mentioned ticker symbols in our Discord data.

In [ ]:
# Extract all ticker mentions
all_tickers = [ticker for tickers_list in clean_df['tickers'] for ticker in tickers_list if tickers_list]

# Count occurrences
ticker_counts = pd.Series(all_tickers).value_counts().reset_index()
ticker_counts.columns = ['ticker', 'mentions']

# Display top 10 tickers
ticker_counts.head(10)

## Sentiment Analysis

Let's visualize the sentiment distribution for messages mentioning ticker symbols.

In [ ]:
# Filter for messages with tickers
msgs_with_tickers = clean_df[clean_df['tickers'].apply(lambda x: len(x) > 0)]

# Plot sentiment distribution
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(msgs_with_tickers['sentiment'], bins=20, kde=True)
plt.title('Sentiment Distribution for Stock-Related Messages')
plt.xlabel('Sentiment Score (-1 to 1)')
plt.ylabel('Count')
plt.axvline(x=0, color='red', linestyle='--')
plt.show()